# Evaluation

## Sober Leaderboard

`input`: 某个model的result folder path

`output`: leaderboard

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
import numpy as np


def summarize_leaderboard_results(result_path, tasks, metric="extractive_match", output="avg.csv"):
    """
    扫描 result_path 下所有 json，计算每个 task 的 mean / std，以及 mean of means，
    并保存到 CSV。

    Args:
        result_path (str / Path): JSON 文件所在目录
        tasks (list[str]): 任务名称列表，如 ["aime24", "aime25", ...]
        metric (str): 要提取的指标 key (默认 extractive_match)
        output (str): 输出 CSV 文件名 (在 result_path 内输出)

    Returns:
        pandas.DataFrame: 汇总后的 DataFrame
    """

    result_path = Path(result_path)
    task_values = {task: [] for task in tasks}

    # 遍历文件夹
    for root, dirs, files in os.walk(result_path):
        for f in files:
            if f.endswith(".json"):
                fpath = os.path.join(root, f)
                with open(fpath, "r") as fp:
                    data = json.load(fp)

                for task in tasks:
                    key = f"custom|{task}|0"
                    if key in data.get("results", {}):
                        value = data["results"][key].get(metric, None)
                        task_values[task].append(value)

    # mean / std 聚合
    mean_dict = {}
    std_dict = {}
    mean_values = []

    for task, values in task_values.items():
        arr = np.array(values, dtype=np.float64)
        task_mean = np.nanmean(arr) * 100
        task_std = np.nanstd(arr) * 100

        mean_dict[task] = round(task_mean, 1)
        std_dict[f"{task}_std"] = round(task_std, 1)
        mean_values.append(task_mean)

    # Mean of Means
    mean_of_means = round(np.nanmean(mean_values), 1)
    result = {**mean_dict, "Avg.": mean_of_means, **std_dict}

    # 列顺序
    mean_cols = tasks
    avg_col = ["Avg."]
    std_cols = [f"{task}_std" for task in tasks]
    ordered_cols = mean_cols + avg_col + std_cols

    df = pd.DataFrame([result], columns=ordered_cols)

    # 输出 CSV
    output_path = result_path / output
    df.to_csv(output_path, index=False)

    print(df)
    print(f"✅ 已保存到 {output_path}")

    return df

df = summarize_leaderboard_results(
    result_path="result/ties_accfmt",
    tasks=["aime24", "aime25", "amc23", "math_500", "minerva", "olympiadbench"],
    metric="extractive_match",
    output="leaderboard.csv"
)

df


In [ ]:
import os
import pandas as pd
from pathlib import Path


def collect_all_results(root_path, tasks, metric="extractive_match", output="all_models.csv"):
    """
    遍历 root_path 下所有子文件夹，对每个子文件夹调用 summarize_leaderboard_results，
    收集多个 DataFrame，并合并成一个总 CSV。

    Args:
        root_path (str / Path): 包含多个 experiment 子目录的根目录
        tasks (list[str]): benchmark list
        metric (str): 提取的指标
        output (str): 输出文件名

    Returns:
        pandas.DataFrame: 合并后的 DataFrame
    """

    root_path = Path(root_path)
    all_dfs = []

    for subdir in sorted(root_path.iterdir()):
        if subdir.is_dir():
            print(f"📂 Processing model: {subdir.name}")
            df = summarize_leaderboard_results(
                result_path=subdir,
                tasks=tasks,
                metric=metric,
                output="leaderboard.csv"
            )
            df["model"] = subdir.name  # 加上模型标识
            all_dfs.append(df)

    # 合并
    final_df = pd.concat(all_dfs, ignore_index=True)

    # 输出 CSV
    output_path = root_path / output
    final_df.to_csv(output_path, index=False)

    print("\n✅ All experiments summarized.")
    print(f"📄 Final CSV saved to: {output_path}")
    print(final_df)

    return final_df


# -------- 运行 --------
collect_all_results(
    root_path="/Users/zhikaili/workspace/sober-reasoning/result/ties_accfmt/results",     # 这个目录里面包含多个子目录，每个子目录对应一个 experiment
    tasks=["aime24", "aime25", "amc23", "math_500", "minerva", "olympiadbench"],
    metric="extractive_match",
    output="all_models.csv"
)


## Plot Eval vs Ratio

input: 包含多个model评估结果的folder，文件夹名字中要带有ratio用于解析
ouput: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# ===============================
# Data
# ===============================

ratios = [
    "0:1",
    "0.1:0.9",
    "0.2:0.8",
    "0.3:0.7",
    "0.4:0.6",
    "0.5:0.5",
    "0.6:0.4",
    "0.7:0.3",
    "0.8:0.2",
    "0.9:0.1",
    "1:0",
]

# Accuracy + Format
acc_fmt = [48.9, 49.0, 49.4, 48.6, 49.2, 49.5, 49.7, 50.5, 48.8, 48.7, 49.1]

# Cosine + Format
cos_fmt = [48.9, 50.1, 48.3, 48.2, 48.6, 49.5, 50.0, 49.9, 49.0, 48.9, 49.5]

# Baseline values
baseline_dra = 49.3
baseline_rs = 49.0
baseline_base = 48.4


# ===============================
# Plot
# ===============================

plt.figure(figsize=(14, 6))

# 两条 main 曲线
plt.plot(ratios, acc_fmt, marker="o", linewidth=2, label="Accuracy + Format")
plt.plot(ratios, cos_fmt, marker="o", linewidth=2, label="Cosine + Format")

# Baseline
plt.axhline(
    baseline_dra,
    linestyle="--",
    linewidth=2,
    color="red",
    label=f"Baseline: DRA-GRPO ({baseline_dra})",
)

plt.axhline(
    baseline_rs,
    linestyle="--",
    linewidth=2,
    color="purple",
    label=f"Baseline: OPEN-RS ({baseline_rs})",
)

plt.axhline(
    baseline_base,
    linestyle="--",
    linewidth=2,
    color="gray",
    label=f"Baseline: Base Model ({baseline_base})",
)

# ===============================
#  Expert 标签标注
# ===============================

# Format Expert → ratio = 0:1
plt.text(0, acc_fmt[0], "Format Expert", fontsize=12, color="black")

# Accuracy Expert → ratio = 1:0（在 acc+format 曲线）
plt.text(10, acc_fmt[-1], "Accuracy Expert", fontsize=12, color="black")

# Cosine Expert → ratio = 1:0（在 cosine+format 曲线）
plt.text(10, cos_fmt[-1], "Cosine Expert", fontsize=12, color="black")


plt.title("Evaluation Performance Across Ratio", fontsize=18)
plt.xlabel("Accuracy/Cosine : Format Ratio", fontsize=14)
plt.ylabel("Average Accuracy", fontsize=14)

plt.xticks(rotation=30)
plt.grid(alpha=0.4, linestyle="--")
plt.legend()

plt.tight_layout()
plt.show()

# Validation

收集所有lr，所有step的validation结果
画出折线图，横坐标是step，纵坐标是val acc，每条折线代表一个lr

## Collect Result

In [ ]:
import os
import json
import pandas as pd
import numpy as np

def find_json_files(path):
    """递归查找 path 下的所有 .json 文件"""
    json_files = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if f.endswith(".json"):
                json_files.append(os.path.join(root, f))
    return json_files


def summarize_results_to_csv(
    root_dir,
    tasks=["aime24", "aime25", "amc23", "math_500", "minerva", "olympiadbench"],
    metrics=["extractive_match"],
    output="results_summary.csv"
):
    """
    汇总 JSON -> CSV

    参数:
    - root_dir: JSON 搜索的根目录
    - tasks:    任务名称列表
    - metrics:  指标 key 列表
    - output:   输出 CSV 文件名称

    返回:
    - df (DataFrame): 汇总结果
    """

    records = {}

    # 找全部 JSON 文件
    json_files = find_json_files(root_dir)
    print(f"🔍 共找到 {len(json_files)} 个 JSON 文件")

    for fpath in json_files:
        with open(fpath, "r") as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"⚠️ 无法读取 {fpath}: {e}")
                continue

        model = os.path.relpath(fpath, root_dir)

        for task in tasks:
            for metric in metrics:
                key = f"{task}_{metric}"
                try:
                    value = data["results"][f"custom|{task}|0"][metric]
                except KeyError:
                    continue

                if model not in records:
                    records[model] = {}
                if key not in records[model]:
                    records[model][key] = []
                records[model][key].append(value)

    # 将 list 转为平均值
    for model, vals in records.items():
        for key in vals:
            if isinstance(vals[key], list):
                records[model][key] = float(np.mean(vals[key]))

    # 转换为 DataFrame
    rows = []
    for model, vals in records.items():
        row = {"model": model}
        for task in tasks:
            for metric in metrics:
                row[f"{task}_{metric}"] = vals.get(f"{task}_{metric}", None)
        rows.append(row)

    df = pd.DataFrame(rows)

    # 每个 metric 单独求平均
    for metric in metrics:
        cols = [f"{task}_{metric}" for task in tasks if f"{task}_{metric}" in df.columns]
        if cols:
            df[f"avg_{metric}"] = df[cols].mean(axis=1, skipna=True)

    # extractive_match ×100
    for col in df.columns:
        if "extractive_match" in col:
            df[col] = df[col].apply(lambda x: round(x * 100, 1) if pd.notnull(x) else x)

    # 保存 CSV
    csv_path = os.path.join(root_dir, output)
    df.to_csv(csv_path, index=False)

    print(f"✅ 已保存 {csv_path}")
    print(f"📊 共汇总 {len(df)} 条记录")

    return df

df = summarize_results_to_csv(
    root_dir="/local/scratch/zli2255/workspace/open-r1/experiments/",
    metrics=["extractive_match"],
    output="summary_extractive.csv"
)


## Parse result

In [ ]:
import pandas as pd
import re
from pathlib import Path

# CSV 文件路径
csv_path = Path('/Users/zhikaili/workspace/open-r1/result/aime2425_acc_len_1e-06/result.csv')
df = pd.read_csv(csv_path)

def extract_info(path: str):
    """
    从路径中提取 ratio, lr, step
    例如：experiments_exp_grpo_fft_ckpt_reward_expert_0001_9e-07_checkpoint-50
    """
    match = re.search(
        r'_([0-9]+)_([\d.eE+-]+)_checkpoint-(\d+)',
        path
    )
    if match:
        ratio = match.group(1)
        lr = match.group(2)
        step = match.group(3)
        return ratio, lr, step
    else:
        return None, None, None

# 提取 ratio、lr、step
df[['ratio', 'lr', 'step']] = df['model'].apply(
    lambda x: pd.Series(extract_info(x))
)

# 转换类型
df['lr'] = df['lr'].astype(float)
df['step'] = df['step'].astype(int)

# 保存修改后的 CSV
df.to_csv(csv_path.parent / 'parsed_result.csv', index=False)
print(f"✅ Updated CSV saved to {csv_path.parent / 'parsed_result.csv'}")

## Aggregate all benchmark

In [ ]:
# 将空值 NaN 转为 0
# df_filled = df.fillna(0)

# 删除 object 类型的列（比如字符串列）
df_numeric = df.drop(columns=['model'])

# 对 ratio, lr, step 分组取平均
df_agg = df_numeric.groupby(['ratio', 'lr', 'step'], as_index=False).mean()

# 保存结果
output_path = csv_path.parent / 'aggregated_result.csv'
df_agg.to_csv(output_path, index=False)
print(f"✅ Aggregated CSV saved to {output_path}")


## Plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取 CSV
df = pd.read_csv(output_path, dtype={'ratio': str})

df['ratio'] = df['ratio'].astype(str)

# 获取所有 learning rates
keep_lrs = [1e-6]
df = df[df['lr'].isin(keep_lrs)]
learningrates = df['lr'].unique()
learningrates.sort()

# 排序 step
df = df.sort_values(by='step')


# 绘图函数
def plot_metric(sub_df, metric, ylabel, title, ratio_val):
    plt.figure(figsize=(10,5))
    for lr in learningrates:
        lr_df = sub_df[sub_df['lr'] == lr]

        if metric == 'avg_extractive_match':
            steps = lr_df['step'].tolist()
            values = lr_df[metric].tolist()
        else:
            steps = lr_df['step'].tolist()
            values = lr_df[metric].tolist()

        plt.plot(steps, values, label=lr)

    plt.xlabel('Step')
    plt.ylabel(ylabel)
    plt.title(f"{title} (ratio={ratio_val})")
    plt.legend(title='Learning Rate')
    plt.grid(True)
    plt.show()

# 对每个 ratio 绘图
for ratio_val, sub_df in df.groupby('ratio'):
    plot_metric(sub_df, 'avg_extractive_match', 'Avg@3', 'Validation Avg@3 vs Step', ratio_val)
    plot_metric(sub_df, 'avg_mean_token_length', 'Mean Output Token Length', 'Mean Output Token Length vs Step', ratio_val)


## Validation Merged Models

In [ ]:
def generate_strings(n_points=30, total=1):
    result = []
    for i in range(n_points):
        left = round(total * i / (n_points - 1), 3)
        right = round(total - left, 3)
        result.append(f'"{0.000:.3f} {left:.3f} {0.000:.3f} {right:.3f}"')
    return result

strings = generate_strings(30)
for s in strings:
    print(s)


parse

In [ ]:
import pandas as pd
from pathlib import Path

# 读取 CSV
result_csv = Path("val_cosfmt.csv")
df = pd.read_csv(result_csv)

output_path = result_csv.parent / "grouped_result.csv"


df['ratio'] = df['model'].str.extract(r'_([0-9.]+)/').astype(float)


# 按 ratio 分组计算均值和标准差
group_stats = df.groupby('ratio').agg(
    math_validation_mean_token_length_mean=('math_validation_mean_token_length', 'mean'),
    math_validation_mean_token_length_std=('math_validation_mean_token_length', 'std'),
    math_validation_extractive_match_mean=('math_validation_extractive_match', 'mean'),
    math_validation_extractive_match_std=('math_validation_extractive_match', 'std')
).reset_index()


group_stats.to_csv(output_path, index=False)

group_stats.head(5)


## Plot Val vs Ratio

In [ ]:

import matplotlib.pyplot as plt

df = pd.read_csv(output_path)  # 替换为你的文件路径

# 图 1: mean_token_length
plt.figure(figsize=(8, 5))
plt.plot(df['ratio'], df['math_validation_mean_token_length_mean'], marker='o')
plt.xlabel('Ratio')
plt.ylabel('Mean Token Length')
plt.title('Mean Token Length vs Merging Ratio')
plt.grid(True)
plt.show()

# 图: math_validation_extractive_match_mean + std (填充带)
plt.figure(figsize=(8,5))
plt.plot(df['ratio'], df['math_validation_extractive_match_mean'], 'o-', color='blue', label='Mean')
plt.fill_between(
    df['ratio'],
    df['math_validation_extractive_match_mean'] - df['math_validation_extractive_match_std'],
    df['math_validation_extractive_match_mean'] + df['math_validation_extractive_match_std'],
    color='blue',
    alpha=0.1,  # 半透明
    label='Std Dev'
)
plt.xlabel('Ratio')
plt.ylabel('Avg@3')
plt.title('Validation Performance vs Merging Ratio')
plt.axhline(y=44, color='red', linestyle='--', linewidth=2, label='Base Model')

plt.text(0.1, df['math_validation_extractive_match_mean'].iloc[-1], "Format Expert",
         ha='center', va='bottom', color='black', fontsize=10)
plt.text(0.9, df['math_validation_extractive_match_mean'].iloc[0], "Cosine Expert",
         ha='center', va='bottom', color='black', fontsize=10)


plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.show()




# Detail -> Acc, Token Length

一个model会进行多次validation，每次validation会有一个对应的.parquet文件，因此对于一个model的一次validation，需要从这些parquet文件中提取acc和len信息。
这里的一个model可以是某个step的checkpoint，我们需要收集所有checkpoint的acc和len，从而构建pareto front。



- `fix` 10个ratio10个step的checkpoint都在aime24上进行十次验证 -> 10 * 10 * (acc, len) -> pareto
- `merge` 同上
- `dynamic` mix **1个ratio**10个step的checkpoint都在aime24上进行十次验证?




In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer


def analyze_parquet_folder(
    folder_path: str,
    task: str,
    model_name: str = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    save_csv: bool = False,
):
    """
    递归遍历指定文件夹下的所有 .parquet 文件，
    计算平均 token length 与 extractive_match。

    参数:
        folder_path (str): 要分析的文件夹路径
        model_name (str): 用于分词的模型名称
        save_csv (bool): 是否保存详细结果 CSV (默认 False)

    返回:
        dict: {"avg_token_length": float, "avg_extractive_match": float}
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # === 收集所有 parquet 文件 ===
    parquet_files = []
    for root, _, files in os.walk(folder_path):
        for f in files:
            if f.endswith(".parquet") and task in f:
                parquet_files.append(os.path.join(root, f))

    if not parquet_files:
        print(f"⚠️ No parquet files found under {folder_path}")
        return {"avg_token_length": np.nan, "avg_extractive_match": np.nan}

    print(f"🔍 Found {len(parquet_files)} parquet files under {folder_path}")

    records = []

    for parquet_path in parquet_files:
        try:
            details = load_dataset("parquet", data_files=parquet_path, split="train")
            token_lengths, extractive_matches = [], []

            for item in details:
                preds = item.get("predictions", [])
                if preds and isinstance(preds[0], str):
                    tokens = tokenizer.encode(preds[0])
                    token_lengths.append(len(tokens))

                metrics = item.get("metrics", {})
                if isinstance(metrics, dict) and "extractive_match" in metrics:
                    extractive_matches.append(metrics["extractive_match"])

            if token_lengths or extractive_matches:
                avg_len = np.mean(token_lengths) if token_lengths else np.nan
                avg_em = np.mean(extractive_matches) if extractive_matches else np.nan
                records.append(
                    {
                        "file": os.path.basename(parquet_path),
                        "avg_token_length": avg_len,
                        "avg_extractive_match": avg_em,
                    }
                )
                print(
                    f"📄 {os.path.basename(parquet_path)} → tokens={avg_len:.1f}, extractive_match={avg_em:.3f}"
                )

        except Exception as e:
            print(f"❌ Failed to process {parquet_path}: {e}")

    if not records:
        print("⚠️ No valid data found.")
        return {"avg_token_length": np.nan, "avg_extractive_match": np.nan}

    df = pd.DataFrame(records)
    avg_len = df["avg_token_length"].mean()
    avg_em = df["avg_extractive_match"].mean()

    print(f"\n✅ Folder summary → tokens={avg_len:.1f}, extractive_match={avg_em:.3f}")

    if save_csv:
        output_csv = os.path.join(folder_path, "aggregated_metrics.csv")
        df.to_csv(output_csv, index=False)
        print(f"💾 Detailed results saved to: {output_csv}")

    return {"avg_token_length": avg_len, "avg_extractive_match": avg_em}


# === 参数设置 ===
parent_dir = "/Users/zhikaili/workspace/open-r1/result/aime25_accfmt/details"
task = "aime25"
output_csv = os.path.join(parent_dir, "summary.csv")
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"


records = []

# === 遍历子文件夹 ===
for folder_name in sorted(os.listdir(parent_dir)):
    folder_path = os.path.join(parent_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue

    print(f"📂 Processing folder: {folder_name}")
    try:
        result = analyze_parquet_folder(folder_path, task, model_name)
        result["folder_name"] = folder_name
        records.append(result)
    except Exception as e:
        print(f"⚠️ Failed to process {folder_name}: {e}")

# === 汇总保存 ===
if records:
    df = pd.DataFrame(records)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Summary saved to {output_csv}")
    print(df)
else:
    print("⚠️ No valid results found.")

## CSV:(step, acc, len) -> Pareto

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === 读取 CSV ===
df = pd.read_csv("result/aime2425_acc_len_1e-06/acclenstep.csv")


# === Step 1: 散点图 ===
plt.figure(figsize=(8,6))
plt.scatter(df['acc'], df['len'], c='blue', label='all points')
plt.xlabel("Accuracy")
plt.ylabel("Token length")
plt.title("Scatter plot of (acc, len)")
plt.grid(True)

# === Step 2: 找 Pareto front ===
points = df[['acc','len']].values

def pareto_front(points):
    """返回布尔 mask，True 表示在 Pareto 前沿"""
    N = points.shape[0]
    mask = np.ones(N, dtype=bool)
    for i in range(N):
        if not mask[i]:
            continue
        for j in range(N):
            if i == j:
                continue
            # j dominates i if acc_j >= acc_i and len_j <= len_i and at least one strict
            if (points[j,0] >= points[i,0] and points[j,1] <= points[i,1]) and (
                points[j,0] > points[i,0] or points[j,1] < points[i,1]
            ):
                mask[i] = False
                break
    return mask

mask = pareto_front(points)
pareto_points = points[mask]

# === Step 3: 绘制 Pareto front ===
plt.scatter(pareto_points[:,0], pareto_points[:,1], c='red', label='Pareto front')
plt.plot(pareto_points[:,0], pareto_points[:,1], c='red', linestyle='--')  # 连线
plt.legend()
plt.show()
